In [1]:
import geopandas as gpd

In [2]:
dispositions = gpd.read_file('data/mineral_disposition/mineral_disposition.shp')

dispo_part = gpd.read_file('data/mineral_disp_partial_cells/mineral_disp_partial_cells.shp')

In [3]:
figh_dispos = dispositions[dispositions.OWNERS.str.contains('Faith')]

In [4]:
figh_dispos_part = dispo_part[dispo_part.OWNERS.str.contains('Faith')]

In [6]:
dispositions.centroid

0       POINT (603048.1877534462 6482803.859342211)
1       POINT (471096.8583876095 6359180.253102098)
2       POINT (213107.7302730975 6447294.200629468)
3       POINT (219905.1391658014 6450825.584254365)
4       POINT (220799.8599835827 6462248.874234542)
                           ...                     
5447     POINT (219992.327856311 6401223.209322074)
5448    POINT (499837.3832856638 6136953.235294266)
5449    POINT (499189.1658359562 6134632.925130632)
5450    POINT (470719.9520846204 6551582.991333066)
5451    POINT (468126.3783594915 6547197.693768341)
Length: 5452, dtype: object

Typically I would go straight to SEDAR to find a NI 43-101 filing for the deposit. But Faith in Gravity is . 

In [18]:
import folium

figh_dispos_wm = figh_dispos.to_crs({'init':'epsg:4326'})

map = folium.Map()

folium.GeoJson(figh_dispos).add_to(map)
folium.GeoJson(figh_dispos_part).add_to(map)

for row in figh_dispos_wm.centroid:
    map.add_child(folium.CircleMarker([row.y, row.x]))

map

Find the k nearest neighbors - kdtree

(The cKDTree is a Cython wrapped C++ implementation of the KDTree that offers some substantial performance benefits.)[https://stackoverflow.com/questions/6931209/difference-between-scipy-spatial-kdtree-and-scipy-spatial-ckdtree0]

In [19]:
from scipy.spatial import cKDTree

not_figh_dispos = dispositions[~dispositions.OWNERS.str.contains('Faith')]

In [34]:
import numpy as np

nfg_centroids = np.vstack((not_figh_dispos.centroid.x.values, not_figh_dispos.centroid.y.values)).T

In [36]:
fg_centroids = np.vstack((figh_dispos.centroid.x.values, figh_dispos.centroid.y.values)).T

In [37]:
nfg_tree = cKDTree(nfg_centroids)

In [40]:
dispos_10km = nfg_tree.query_ball_point(fg_centroids, r = 10000)

In [54]:
import itertools

dispos_10km_index = list(set(itertools.chain.from_iterable(dispos_10km)))

dispos_10km_index

[3204,
 3206,
 3209,
 2575,
 3354,
 2626,
 3358,
 3359,
 3360,
 3361,
 3362,
 3363,
 3364,
 2600,
 2604,
 2611,
 4157,
 2614,
 2488,
 2489,
 2490,
 2491,
 2492,
 2493,
 2494,
 2495,
 2496,
 2497,
 2498,
 2499,
 2500,
 2501,
 582,
 583,
 2502,
 2503,
 2504,
 2505,
 2634,
 2510,
 2512,
 2513,
 2514,
 2515,
 2516,
 2519,
 2534,
 3815,
 2535,
 2537,
 3818,
 2667,
 2540,
 3821,
 2668,
 2669,
 755,
 2618,
 2550,
 2619]

In [56]:
fig_neighbors = not_figh_dispos.iloc[dispos_10km_index]

In [65]:
black_lake_coords = [59.51684, -105.26733]

map = folium.Map(location=black_lake_coords, zoom_start = 9)

folium.GeoJson(figh_dispos).add_to(map)
folium.GeoJson(figh_dispos_part).add_to(map)
folium.GeoJson(fig_neighbors, style_function=lambda feature: {
        'fillColor': '#FF0000',
        'color' : '#FF0000',
        'weight' : 1,
        'fillOpacity' : 0.5,
        }).add_to(map)

map

What's in a name? - how anomalous is the gravity anomaly? 